In [220]:
string  ="""
BULK
 Item Id   Number       Status    Weight         Pieces    Volume    Origin    Onl.Sta.  Dest.     Category  Position  Confirmed  Remarks  
 37144805  NULL         ESTIMATE  0.00      KG   0         0.00      DUB       DUB       EDI       B         5         false     NULL 
ULD
 Item Id   ULD       Number    Airline   Special   Status    Weight         Pieces    Volume    Origin    Onl.Sta.  Dest.     Category  Position  Confirmed  Remarks  
 37137862  AKH       NULL      MN        NULL      ESTIMATE  88.00     KG   0         0.00      DUB       DUB       EDI       X         11        false     NULL 
 37137860  AKH       NULL      MN        NULL      ESTIMATE  88.00     KG   0         0.00      DUB       DUB       EDI       X         12        false     NULL 
 37137858  AKH       NULL      MN                  ESTIMATE  351.00    KG   27        0.00      DUB       DUB       EDI       B         41        false     NULL 
 37137861  AKH       NULL      MN        NULL      ESTIMATE  88.00     KG   0         0.00      DUB       DUB       EDI       X         13        false     NULL 
 37137857  AKH       NULL      MN                  ESTIMATE  455.00    KG   35        0.00      DUB       DUB       EDI       B         42        false     NULL 
STATUS FUEL 2 AIRCRAFT_CONFIG 1 EZFW 1 CARGO_TRANSFER 1 OFP 4 CABIN_CONFIG 1 AUTO_MODE_ACTIVE 1 AUTOMATION_STARTED 0 BAG_LOAD_ITEMS_GEN 1 EZFW_COUNTER 1 REGISTRATION 1 REGISTRATION_CHANGE 1 FUEL_ORDER 2",f506e2277d8b1ff7,Saved
"""

In [221]:
import re

# find "ULD" in the string using regex
match = re.search(r'ULD', string)
new_string = ""
if match:
    # create a new string from the matched position until the end
    new_string += string[match.start():]
new_string = new_string.split("\n")
lines = new_string[1:-2]

In [222]:
lines

[' Item Id   ULD       Number    Airline   Special   Status    Weight         Pieces    Volume    Origin    Onl.Sta.  Dest.     Category  Position  Confirmed  Remarks  ',
 ' 37137862  AKH       NULL      MN        NULL      ESTIMATE  88.00     KG   0         0.00      DUB       DUB       EDI       X         11        false     NULL ',
 ' 37137860  AKH       NULL      MN        NULL      ESTIMATE  88.00     KG   0         0.00      DUB       DUB       EDI       X         12        false     NULL ',
 ' 37137858  AKH       NULL      MN                  ESTIMATE  351.00    KG   27        0.00      DUB       DUB       EDI       B         41        false     NULL ',
 ' 37137861  AKH       NULL      MN        NULL      ESTIMATE  88.00     KG   0         0.00      DUB       DUB       EDI       X         13        false     NULL ',
 ' 37137857  AKH       NULL      MN                  ESTIMATE  455.00    KG   35        0.00      DUB       DUB       EDI       B         42        false     NULL ']

In [223]:
dummy_row = "37137862  AKH       NULL      MN        NULL      ESTIMATE  88.00     KG   0         0.00      DUB       DUB       EDI       X         11        false     NULL"
dummy_row = dummy_row.replace("KG", "")
# split the string into words
words = dummy_row.split()

# initialize lists to store indices and first characters
indices_uld = []

# track the current index in the original string
current_index = 0

# iterate through each word
for word in words:
    # get the first character of the word
    first_char = word[0]
    # find the index of the first character in the original string
    index = dummy_row.index(first_char, current_index)
    # append the index and character to the lists
    indices_uld.append(index)
    # update the current index to search for the next word
    current_index = index + len(word)


In [224]:
import re
data = []
# allColumns = Item Id   ULD       Number    Airline   Special   Status    Weight         Pieces    Volume    Origin    Onl.Sta.  Dest.     Category  Position  Confirmed  Remarks 
columns = re.sub(r'\s+', ',', lines[0].lstrip().replace("Item Id", "ItemID"))[:-1].split(",")

for line in lines[1:]:
    line = line.lstrip()
    line = line.replace("KG","")
    dummy_row_list = list(line)
    for index in indices_uld:
        if dummy_row_list[index] == " ":
            # replace the character and the following 3 characters with "NULL"
            dummy_row_list[index:index+4] = "NULL"

    # convert the list back to a string
    line = ''.join(dummy_row_list)

    line = re.sub(r'\s+', ',', line)
    line = line.split(",")
    dictionary = dict(zip(columns, line))
    data.append(dictionary)




In [225]:
df = pd.DataFrame(data)
df

,ItemID,ULD,Number,Airline,Special,Status,Weight,Pieces,Volume,Origin,Onl.Sta.,Dest.,Category,Position,Confirmed,Remarks
0,37137862,AKH,NULL,MN,NULL,ESTIMATE,88.00,0,0.00,DUB,DUB,EDI,X,11,false,NULL
1,37137860,AKH,NULL,MN,NULL,ESTIMATE,88.00,0,0.00,DUB,DUB,EDI,X,12,false,NULL
2,37137858,AKH,NULL,MN,NULL,ESTIMATE,351.00,27,0.00,DUB,DUB,EDI,B,41,false,NULL
3,37137861,AKH,NULL,MN,NULL,ESTIMATE,88.00,0,0.00,DUB,DUB,EDI,X,13,false,NULL
4,37137857,AKH,NULL,MN,NULL,ESTIMATE,455.00,35,0.00,DUB,DUB,EDI,B,42,false,NULL


In [226]:
import re

# Find "ULD" in the string using regex
match = re.search(r'BULK', string)
new_string = ""
if match:
    # Create a new string from the matched position until the end
    new_string += string[match.start():]
new_string2 = new_string.split("\n")


In [227]:
result = list(set(new_string2) - set(lines))
print(result)
result = result[1:]
result = [result[0]] +  result[2:]
result = result[:-1]
result = [result[0]] + result[2:]
lines = result

['', ' Item Id   Number       Status    Weight         Pieces    Volume    Origin    Onl.Sta.  Dest.     Category  Position  Confirmed  Remarks  ', 'STATUS FUEL 2 AIRCRAFT_CONFIG 1 EZFW 1 CARGO_TRANSFER 1 OFP 4 CABIN_CONFIG 1 AUTO_MODE_ACTIVE 1 AUTOMATION_STARTED 0 BAG_LOAD_ITEMS_GEN 1 EZFW_COUNTER 1 REGISTRATION 1 REGISTRATION_CHANGE 1 FUEL_ORDER 2",f506e2277d8b1ff7,Saved', 'BULK', ' 37144805  NULL         ESTIMATE  0.00      KG   0         0.00      DUB       DUB       EDI       B         5         false     NULL ', 'ULD']


In [228]:
lines

[' Item Id   Number       Status    Weight         Pieces    Volume    Origin    Onl.Sta.  Dest.     Category  Position  Confirmed  Remarks  ',
 ' 37144805  NULL         ESTIMATE  0.00      KG   0         0.00      DUB       DUB       EDI       B         5         false     NULL ']

In [229]:
dummy_row = "37144805  NULL         ESTIMATE  0.00      KG   0         0.00      DUB       DUB       EDI       B         5         false     NULL"
dummy_row = dummy_row.replace("KG", "")
# Split the string into words
words = dummy_row.split()

# Initialize lists to store indices and first characters
indices_bulk= []

# Track the current index in the original string
current_index = 0

# Iterate through each word
for word in words:
    # Get the first character of the word
    first_char = word[0]
    # Find the index of the first character in the original string
    index = dummy_row.index(first_char, current_index)
    # Append the index and character to the lists
    indices_bulk.append(index)
    # Update the current index to search for the next word
    current_index = index + len(word)


In [230]:
import re
#allColumns = Item Id   ULD       Number    Airline   Special   Status    Weight         Pieces    Volume    Origin    Onl.Sta.  Dest.     Category  Position  Confirmed  Remarks 
columns = re.sub(r'\s+', ',', lines[0].lstrip().replace("Item Id", "ItemID"))[:-1].split(",")

for line in lines[1:]:
    line = line.lstrip()
    line = line.replace("KG","")
    dummy_row_list = list(line)
    for index in indices_bulk:
        if dummy_row_list[index] == " ":
            # Replace the character and the following 3 characters with "NULL"
            dummy_row_list[index:index+4] = "NULL"

    # Convert the list back to a string
    line = ''.join(dummy_row_list)

    line = re.sub(r'\s+', ',', line)
    line = line.split(",")
    dictionary = dict(zip(columns, line))
    data.append(dictionary)




In [231]:
df = pd.DataFrame(data)
df

,ItemID,ULD,Number,Airline,Special,Status,Weight,Pieces,Volume,Origin,Onl.Sta.,Dest.,Category,Position,Confirmed,Remarks
0,37137862,AKH,NULL,MN,NULL,ESTIMATE,88.00,0,0.00,DUB,DUB,EDI,X,11,false,NULL
1,37137860,AKH,NULL,MN,NULL,ESTIMATE,88.00,0,0.00,DUB,DUB,EDI,X,12,false,NULL
2,37137858,AKH,NULL,MN,NULL,ESTIMATE,351.00,27,0.00,DUB,DUB,EDI,B,41,false,NULL
3,37137861,AKH,NULL,MN,NULL,ESTIMATE,88.00,0,0.00,DUB,DUB,EDI,X,13,false,NULL
4,37137857,AKH,NULL,MN,NULL,ESTIMATE,455.00,35,0.00,DUB,DUB,EDI,B,42,false,NULL
5,37144805,NaN,NULL,NaN,NaN,ESTIMATE,0.00,0,0.00,DUB,DUB,EDI,B,5,false,NULL
